### Make a list of all the image file names, that are saved locally:

In [3]:
from os import listdir
from os.path import isfile, join
mypath = '../../aerial/datasets/s2cloudless_imagery/data'
image_file_names_list_0 = [f for f in listdir(mypath) if isfile(join(mypath, f))]
len(image_file_names_list_0)

282

In [ ]:
https://raw.githubusercontent.com/od-crypto/somedata/master/img/bolivia_popoo_s2cloudless_2016.jpg
    
https://github.com/od-crypto/somedata/tree/master/img

### make_image_urls: 
### makes a list with url-links to the corresponding image file names (the same are on github)

In [4]:
import json

config_name = 'toloka_config.json'
with open(config_name, 'r') as f:
    config = json.load(f)

def make_image_urls(image_file_names_list_0):
    #image_list = IMAGE_URLS_HEADER
    image_urls_list_0 = []
    for f in image_file_names_list_0:
        f = config["link"] + f   
        image_urls_list_0.append(f)
    return image_urls_list_0

### apply the function to create the url-links list for the Toloka task:

In [5]:
image_urls_list_0 = make_image_urls(image_file_names_list_0);
image_urls_list_0;

### Function tsv-toloka creates a tsv file with the task for Toloka:

In [6]:
#number_lakes_toloka = ["GOLDEN:lakes"]
#number_islands_toloka = ["GOLDEN:islands"]

IMAGE_URLS_HEADER = ["INPUT:image"]
IMAGE_FILE_NAMES_HEADER = ["INPUT:name"]
RESULT_HEADER = ["GOLDEN:result"]

HINT_HEADER = ["HINT:text"]

def tsv_toloka(image_urls_list_0, IMAGE_URLS_HEADER, RESULT_HEADER, HINT_HEADER, output_file_path, IMAGE_FILE_NAMES_HEADER=None):

    numItems = len(image_urls_list_0)
    outputStr = ''

    image_urls_list = IMAGE_URLS_HEADER + image_urls_list_0
    
    image_file_names_list_0 = [i.split('/')[-1] for i in image_urls_list_0]
    image_file_names_list = IMAGE_FILE_NAMES_HEADER + image_file_names_list_0
    
    result = RESULT_HEADER + ['']*(numItems)
    hint = HINT_HEADER + ['']*(numItems)

    for i in range(0, numItems+1): 
        outputStr += image_urls_list[i] + "\t" + image_file_names_list[i] + "\t" + result[i] + "\t" + hint[i] + "\n"
        # outputStr += files_toloka[i] + "\n"


    with open(output_file_path, "w") as outputter:
        outputter.write(outputStr)
        
    df = pd.read_csv(output_file_path, sep='\t')
    
    return df.info()

In [18]:
import pandas as pd
import csv
tsv_toloka(image_urls_list_0, IMAGE_URLS_HEADER, RESULT_HEADER, HINT_HEADER, 'Toloka_task_0.tsv', IMAGE_FILE_NAMES_HEADER)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   INPUT:image    282 non-null    object 
 1   INPUT:name     282 non-null    object 
 2   GOLDEN:result  0 non-null      float64
 3   HINT:text      0 non-null      float64
dtypes: float64(2), object(2)
memory usage: 8.9+ KB


# Read Toloka result tsv files into pandas

In [8]:
import pandas as pd

# Labeling results from the first Toloka task
accepted = "assignments_from_pool_16397010__10-09-2020_accepted.tsv"
df_accepted = pd.read_csv(accepted, sep='\t')

# Labeling results from the second Toloka task: 
check = "assignments_from_pool_16483821__15-09-2020_check_29.tsv"
df_check = pd.read_csv(check, sep='\t')

### Manually collected images from Toloka tasks to be resent: 

In [10]:
remove_from_accepted = config['remove_from_accepted']
remove_from_task2 = config['remove_from_task2']
# incorrect links in the second Toloka task:
broken_task2 = config['broken_task2']

accepted_results = df_accepted.loc[:,['OUTPUT:result']]
accepted_images = df_accepted.loc[:,'INPUT:image'].to_list()
check_images = df_check.loc[:,'INPUT:image'].to_list()

print("remove manually from accepted (1st task): ", len(remove_from_accepted))
print("accepted_images (1st task): ", len(df_accepted), len(accepted_images), len(set(accepted_images)) )
print("check_images (2nd task): ", len(df_check), len(check_images), len(set(check_images)) )

remove_check2 = remove_from_task2 + broken_task2
print(len(df_check), len(set(remove_check2)) )

print("Total good images: ", len(df_accepted) - len(set(remove_from_accepted)) + len(df_check) - len(set(remove_check2)))

remove manually from accepted (1st task):  53
accepted_images (1st task):  195 195 195
check_images (2nd task):  86 86 86
86 17
Total good images:  212


In [13]:
df_good2 = df_check[~df_check['INPUT:image'].isin(remove_check2)]

good_list2 = df_good2.loc[:,'INPUT:image'].to_list()
print("Items in good_list2: ", len(df_good2), len(set(good_list2)))

set(remove_check2) - set(check_images)

Items in good_list2:  69 69


set()

In [14]:
df_good1 = df_accepted[~df_accepted['INPUT:image'].isin(remove_from_accepted)]

good_list1 = df_good1.loc[:,'INPUT:image'].to_list()
print("Items in good_list1 (1st task): ", len(df_good1), len(set(good_list1)))

print(len(df_accepted)-len(remove_from_accepted), len(set(remove_from_accepted) - set(accepted_images)))

set(remove_from_accepted) - set(accepted_images)

Items in good_list1 (1st task):  143 143
142 0


set()

## Create the masks and the collages for the processed in Toloka lakes: 

In [10]:
#! pip install demjson

In [11]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [15]:
#from PIL import Image, ImageDraw
import requests
import numpy as np
from urllib.request import urlopen
#from StringIO import StringIO
from io import StringIO
from io import BytesIO
import PIL.Image
import cv2
import os

from create_masks import create_poly, plot_locally, create_collage, create_masks

df_final = pd.concat([df_good1, df_good2])
print(len(df_final))

df = df_final
#create_masks(df, "sentinel_masks", "sentinel_collages", "sentinel_images", False, 0, len(df))

212


## Create new Toloka task file: 

In [16]:
new_task_list = list(set(image_urls_list_0)-set(df_final.loc[:,'INPUT:image'].to_list()))
print(len(new_task_list))

tsv_toloka(new_task_list, IMAGE_URLS_HEADER, RESULT_HEADER, HINT_HEADER, 'Toloka_task.tsv', IMAGE_FILE_NAMES_HEADER)

70
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   INPUT:image    70 non-null     object 
 1   INPUT:name     70 non-null     object 
 2   GOLDEN:result  0 non-null      float64
 3   HINT:text      0 non-null      float64
dtypes: float64(2), object(2)
memory usage: 2.3+ KB
